In [1]:
# 4
import numpy as np
from scipy import stats
weeks_to_death = np.array((49, 58, 75, 110, 112, 132, 151, 276, 281, 362))
mean_death = 200
print "p-value = %.4f" % stats.wilcoxon(weeks_to_death - mean_death).pvalue  # 0.2845

p-value = 0.2845


In [4]:
# 5
no_cut = np.array((22,22,15,13,19,19,18,20,21,13,13,15))
cut = np.array((17,18,18,15,12,4,14,15,10))
print "p-value = %.4f" % stats.mannwhitneyu(cut, no_cut, alternative="less").pvalue  # 0.0290

p-value = 0.0290


In [5]:
# 6
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [6]:
import pandas as pd
np.random.seed(0)

challenger = pd.read_csv("challenger.txt", sep="\t")

In [7]:
challenger.head()

,Unnamed: 0,Temperature,Incident
0,Apr12.81,18.9,0
1,Nov12.81,21.1,1
2,Mar22.82,20.6,0
3,Nov11.82,20.0,0
4,Apr04.83,19.4,0


In [8]:
inc = challenger[challenger.Incident==1]
no_inc = challenger[challenger.Incident==0]
bootstrap_inc_means = map(np.mean, get_bootstrap_samples(np.array(inc.Temperature), 1000))
bootstrap_no_inc_means = map(np.mean, get_bootstrap_samples(np.array(no_inc.Temperature), 1000))

interval = stat_intervals(np.array(bootstrap_inc_means)-np.array(bootstrap_no_inc_means), 0.05)
print "95%% confidence interval: [%.4f, %.4f]" % (interval[0], interval[1])  # -1.4504

95% confidence interval: [-8.0646, -1.4504]


In [9]:
# 7
np.random.seed(0)
import itertools

In [10]:
def permutation_t_stat_ind(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)

In [11]:
def get_random_combinations(n1, n2, max_combinations):
    index = range(n1 + n2)
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        np.random.shuffle(index)
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]

In [12]:
def permutation_zero_dist_ind(sample1, sample2, max_combinations = None):
    joined_sample = np.hstack((sample1, sample2))
    n1 = len(sample1)
    n = len(joined_sample)
    
    if max_combinations:
        indices = get_random_combinations(n1, len(sample2), max_combinations)
    else:
        indices = [(list(index), filter(lambda i: i not in index, range(n))) \
                    for index in itertools.combinations(range(n), n1)]
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    return distr

In [13]:
def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample, mean)
    
    zero_distr = permutation_zero_dist_ind(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)
    

In [14]:
print "p-value =", permutation_test(inc.Temperature, no_inc.Temperature, max_permutations = 10000, 
                                    alternative="two-sided")  # 0.0057

p-value = 0.0057
